<a href="https://colab.research.google.com/github/anthonyhu25/Bayesian-Queuing-/blob/main/Bayesian_Queueing_Inference_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy import random
from numpy import linalg
import math
import scipy
import scipy.stats
import matplotlib.pyplot as plt
from scipy.stats import rv_continuous, rv_discrete
from scipy.stats._distn_infrastructure import rv_frozen
from scipy.special import logsumexp
import warnings
import sys
import statistics
import pandas as pd
from IPython.display import display, Math, HTML
import warnings
warnings.filterwarnings("ignore")

#1. Introduction


For the $M/G/1$ queueing model, customers arrive with independent interarrival times $W_{i} \sim Exp(\theta_{3}) = \theta_{3}e^{-\theta_{3}w_{i}}$

Customers have a service time $U_{i} \sim Uniform(\theta_{1}, \theta_{2})$ distribution. We only observe interdeparture times $Y_{i}$. Purpose of model is to infer the parameters $\theta = (\theta_{1}, \theta_{2}, \theta_{3})$ from the observed interdeparture times $y = (y_{1},...,y_{n})$.

We can represent the interdeparture times $Y_{i}$ as:

$\large Y_{i} = U_{i} + max(0, \sum_{j=1}^{i}W_{j} - \sum_{j=1}^{i-1}Y_{j}) = U_{i} + max(0, V_{i}-X_{i-1})$

where $X_{i} = \sum_{j=1}^{i}$ is the departure time of $i$-th customer, $V_{i}= \sum_{j=1}^{i}W_{j}$ is the arrival time of $i$-th customer, and $W_{j}$ difference in time between arrival times of $j$-th customer and $(j-1)$-th customer.

In the paper, they view the arrival times $V_{i}$ as latent(unobserved) variables, and summarized the conditional distributions with regards to previous arrival times and departure times. But, we need to sample $V_{i}$ conditional on the observed interdepature times $y$, and the previous $V_{i}$ samples (we denote this as $V_{-i}$)

We sample the queuing model $V_{i}$ as follows:

$V_{1} \sim Exp(\theta_{3})$

$V_{i}|V_{i-1} \sim V_{i-1} + Exp(\theta_{3})$

$Y_{i}|X_{i}, V_{i} \sim Uniform(\theta_{1} + max(0, V_{i} - X_{i-1}), \theta_{2} + max(0, V_{i} - X_{i-1})$ for $ i = 1,...,n$

## 2.1 Gibbs update for arrival time

To sample the $V_{i}$, we will use standard Gibbs sampling updates, which will need the conditional density of $V_{i}$ given other $V_{i}$-s and $\theta$.

There are 3 cases presented in the paper: when $i = 1$, $2 \le i \le n-1$, and $i = n$

### Case 1: $i = 1$

$\large P(v_{1}|v_{-1}, y, θ) ∝ P(v_{1})P(y_{1}|v_{1},θ)P(v_{2}|v_{1},θ)$

$\large ∝ [\theta_{3}exp(-\theta_{3}v_{1})I(v_{1} \ge 0)]\frac{I(y_{1} ∈ [\theta_{1} + v_{1}, \theta_{2} + v_{1}])}{\theta_{2}-\theta_{1}}[\theta_{3}exp(-\theta_{3}(v_{2}-v_{1}))I(v_{1} \le v_{2})]$

$\large \propto I(v_{1} \in [max(0, x_{1} - \theta_{1}), min(v_{2}, x_{1} - \theta_{1})] \sim Uniform(max(0, x_{1} - \theta_{1}), min(v_{2}, x_{1} - \theta_{1}))$

Note that the last line comes from rewriting the inequalities $\theta_{1} + v_{1} \le y_{1} \le \theta_{2} + v_{1}$ in terms of $v_{1}$, and also taking into consideration of the bounds $I(v_{1} \ge 0)$ and $I(v_{1} \le v_{2})$

### Case 2: $2 \le i \le n - 1$

Note that $V_{i} - X_{i-1}$ is the time between the arrival of the ${i}$-th customer, and the departure of the $(i-1)$-th customer. So we are sampling interdeparture time $Y_{i}$, conditional on the arrival $V_{i}$ and $X_{i-1}$, and the service time in one step for the uniform service distribution.

Intuitively speaking, we know the arrival times $V_{i-1}$ and $V_{i+1}$, as well as all of the interdeparture times $Y_{i}$ and parameters $\theta$. So we expres probability of $V_{i}$ in terms of the service distribution, shifted based on the location of $V_{i-1}$ and $V_{i+1}$.

$\large P(v_{i}|v_{-i}, y, θ) ∝ P(v_{i}|v_{i-1}, θ)P(y_{i}|x_{i-1}, v_{i}, \theta)P(v_{i+1}|v_{i}, θ)$

$\large \propto \theta_{3}exp(-\theta_{3}(v_{i}-v_{i-1}))I(v_{i-1} \le v_{i})[\frac{y_{i} \in [\theta_{1} + max(0, v_{i}-x_{i-1}), \theta_{2} + max(0, v_{i} - x_{i-1})]}{\theta_{2} - \theta_{1}}] \theta_{3}exp(-\theta_{3}(v_{i+1}-v_{i}))I(v_{i} \le v_{i+1})$

$\large \propto I(v_{i} \in [v_{i-1}, v_{i+1}])I(y_{i} \in [\theta_{1} + max(0, v_{i}-x_{i-1}), \theta_{2} + max(0, v_{i} - x_{i-1})])$

For this expression, we consider two cases: $Y_{i} > \theta_{2}$, and $Y_{i} \le \theta_{2}$, since we seek to establish the behavior of $Y_{i}$ conditional on the service distribution $U_{i}$, and as mentioned before $Y_{i} = U_{i} + max(0, V_{i} - X_{i-1})$

Since $U_{i} \sim Uniform(\theta_{1}, \theta_{2})$, $U_{i} \le \theta_{2}$ for all i. Hence, we examine $Y_{i}$ based on $\theta_{2}$.

If $Y_{i} > \theta_{2}$, this implies that $V_{i} > X_{i-1}$. So, $Y_{i} \sim Uniform(X_{i} - \theta_{1}, max(V_{i+1}, X_{i}-\theta_{2}))$ by rewriting the uniform distribution, which was in terms of $Y_{i}$, into terms of $V_{i}$.

If $Y_{i} \le \theta_{2}$, we split the indicator into regions based on whether $v_{i}$ is lesss than or equal to $x_{i}$. The final uniform distribution simplifies to $I(v_{i} \in [v_{i-1}, min(v_{i+1}, x_{i} - \theta_{1})])$

So for both cases, the arrival time of $i$-th customer $V_{i} \sim Uniform(v_{i-1}, min(v_{i+1}, x_{i} - \theta_{1}))$.

### Case 3: $i = n$

$\large P(v_{n}|v_{-n}, y, \theta) \propto P(v_{n}|v_{n-1})P(y_{n}|v_{n})$

$\propto exp(-\theta_{3}(v_{n}-v_{n-1}))I(v_{n-1} \le v_{n})I(y_{n} \in [\theta_{1} + max(0, v_{n}-x_{n-1}), \theta_{2} + max(0, v_{n} - x_{n-1})])$

If $y_{n} > \theta_{2}$, then $V_{n}$ will have a truncated Exponential distribution on $[x_{n} - \theta_{2}, x_{n} - \theta_{1}]$. Else, $V_{n}$ will have a truncated Exponential distribution on $[v_{n-1}, x_{n}-\theta_{1}]$. In either case, $V_{n}$ has rate $\theta_{3}$.

In [30]:
def gibbs_sampler_queue(y, n, v, theta_dict):
  # length of y vector and v vector must be equal to n
  if len(y) != n:
    raise AttributeError("Length of y-vector is not equal to n")
  if len(v) != n:
    raise AttributeError("Length of v-vector is not equal to n")
  samples = []
  for i in range(n):
    x_i = np.sum(y[:(i+1)])
    y_i = y[i]
    if i == 0:
      lower = max(0, x_i - theta_dict['theta_2'])
      upper = min(v[0], x_i - theta_dict['theta_1'])
      posterior_sample = scipy.stats.uniform(loc = lower, scale = upper - lower).rvs(size = 1)
    elif i == n-1:
      if y_i > theta_dict['theta_2']:
        lower = x_i - theta_dict['theta_2']
        upper = x_i - theta_dict['theta_1']
      else:
        lower = v[i-1]
        upper = float(x_i - theta_dict['theta_1'])
      posterior_sample = scipy.stats.truncexpon(b = theta_dict['theta_3'], loc = lower, scale = upper - lower).rvs(size = 1)
    else:
      lower = v[i-1]
      upper = min(v[i+1], x_i - theta_dict['theta_1'])
      posterior_sample = scipy.stats.uniform(loc = lower, scale = upper - lower).rvs(size = 1)
    samples.append(posterior_sample)
  return np.concatenate(samples).ravel().tolist()

In [3]:
def simulate_v(theta_dict, N, y):
  v_sim = []
  y_sim = []
  for i in range(N):
    X_prev = sum(y[:(i)])
    if len(v_sim) == 0:
      V_i = scipy.stats.expon.rvs(scale = theta_dict['theta_3'], size = 1)
      v_sim.append(float(V_i))
    else:
      V_prev = v_sim[i-1]
      V_i = scipy.stats.expon.rvs(scale = theta_dict['theta_3'], size = 1) + V_prev
      v_sim.append(float(V_i))
    lower = theta_dict['theta_1'] + max(0, V_i - X_prev)
    upper = theta_dict['theta_2'] + max(0, V_i - X_prev)
    Y_i = scipy.stats.uniform.rvs(loc = lower, scale = upper - lower, size = 1)
    y_sim.append(float(Y_i))
  return v_sim, y_sim

In [4]:
def simulate_v_np(theta_dict, N, y):
  v_sim = []
  y_sim = []
  for i in range(N):
    X_prev = sum(y[:(i)])
    if len(v_sim) == 0:
      V_i = np.random.exponential(scale = theta_dict['theta_3'], size = 1)
      v_sim.append(float(V_i))
    else:
      V_prev = v_sim[i-1]
      V_i = np.random.exponential(scale = theta_dict['theta_3'], size = 1) + V_prev
      v_sim.append(float(V_i))
    lower = theta_dict['theta_1'] + max(0, V_i - X_prev)
    upper = theta_dict['theta_2'] + max(0, V_i - X_prev)
    Y_i = scipy.stats.uniform.rvs(loc = lower, scale = upper - lower, size = 1)
    y_sim.append(float(Y_i))
  return v_sim, y_sim

In [5]:
theta_intermediate = {
    'theta_1': 4,
    'theta_2' :7,
    'theta_3': 0.15
}

In [6]:
Frequent = [
11.57, 13.44, 13.24, 9.30, 8.95, 11.99, 15.68, 10.72, 12.68, 9.79,
14.01, 10.04, 12.05, 13.59, 15.13, 15.67, 12.38, 9.11, 9.19, 10.06,
14.73, 10.03, 14.51, 9.95, 15.43, 10.80, 9.57, 10.01, 12.93, 11.79,
10.81, 14.65, 12.68, 12.40, 15.34, 10.29, 14.06, 14.03, 11.04, 12.54,
8.61, 8.43, 12.25, 14.23, 15.47, 9.04, 12.55, 11.76, 8.10, 10.70
]

Intermediate = [
6.19, 6.04, 9.52, 4.49, 4.36, 9.86, 9.91, 5.02, 5.76, 4.67,
6.25, 4.77, 5.52, 6.10, 6.67, 6.88, 5.64, 4.42, 4.45, 4.77,
6.52, 4.76, 6.44, 4.73, 6.79, 5.05, 4.59, 4.75, 5.85, 5.42,
5.05, 6.49, 5.76, 8.67, 16.65, 4.86, 6.27, 6.26, 5.14, 10.60,
4.23, 6.15, 5.59, 6.34, 6.80, 4.39, 5.71, 5.41, 4.04, 5.01
]

Rare = [
21.77, 10.30, 206.34, 8.57, 45.79, 233.13, 128.30, 59.73, 4.59, 3.21,
185.29, 2.49, 4.63, 72.48, 22.47, 195.34, 85.92, 8.39, 23.30, 4.24,
42.78, 332.64, 16.91, 6.26, 39.44, 27.16, 29.53, 93.65, 42.60, 176.36,
34.69, 345.20, 128.16, 307.50, 233.54, 18.79, 36.88, 114.85, 4.73, 337.02,
81.89, 96.33, 27.20, 23.16, 167.89, 70.58, 81.28, 43.55, 33.88, 28.47
]

In [24]:
# Testing intermediate queue
v_sim, y_sim = simulate_v(theta_intermediate, len(Intermediate), Intermediate)
print(v_sim)
print(y_sim)
x_sim = np.cumsum(y_sim)
print(x_sim)
print(np.cumsum(Intermediate))

[0.07207929161351893, 0.10636811310372865, 0.10693095619427952, 0.7959756952901874, 0.8569339704532485, 0.8933229472094723, 0.9758552560060917, 1.0567207145647148, 1.0615694523072148, 1.0790860666714488, 1.3175160445240268, 1.3838583161666234, 1.4660288038273444, 1.5341260481033228, 2.051208438468435, 2.182801606339254, 2.236643190443004, 2.284390325959457, 2.469577019175084, 2.626853379369805, 2.7494218225456972, 2.871639592608981, 3.0138823645646124, 3.165913617286369, 3.3745262587188227, 3.38934505044348, 3.4889854652786365, 3.5526952487357435, 3.9497259808686587, 4.227949276341514, 4.256305620038331, 4.28462816733829, 4.3197448201750515, 4.529799411980133, 4.705576808743802, 4.74370195384865, 4.747591349778898, 4.753538922235982, 4.776456734024684, 4.778658061944093, 4.788822165786059, 4.842928283151696, 5.060331758510461, 5.260356788657536, 5.426533149673254, 5.639176880636097, 5.664718618858208, 5.709272226384698, 6.154570862826282, 6.477622979277413]
[4.627185983395706, 6.402574

In [38]:
gibbs_intermediate = gibbs_sampler_queue(y_sim, len(Intermediate), v_sim, theta_intermediate)
print(gibbs_intermediate)
print(np.cumsum(v_sim))
print(np.cumsum(gibbs_intermediate))
print(x_sim)

[0.027513807000208414, 0.08961895676621945, 0.15678845446428083, 0.40146807088237924, 0.8451347190442997, 0.8949873791547872, 1.0063179980396468, 1.0361569818185006, 1.0647443213735497, 1.2765132189692123, 1.2809393892311893, 1.3863147117678372, 1.4927832690544738, 1.6348949555879355, 1.9104673833597445, 2.1655490893735503, 2.2254579374254613, 2.447021286269193, 2.3004191478843583, 2.688744348105455, 2.7189583157041914, 2.9748672329797254, 2.930820754791556, 3.1774643367370454, 3.2526912115099083, 3.4651446218781343, 3.413829750962684, 3.801453849036447, 3.6099611014307325, 4.248319188571694, 4.249534123353854, 4.259947928538982, 4.292677289882126, 4.580192441540581, 4.727592253589826, 4.718887285436761, 4.749859307994559, 4.773328258704612, 4.765059639758043, 4.777677627235748, 4.822451223800635, 4.873525315394603, 5.209503682000397, 5.319358317058125, 5.62596384717439, 5.537390520394017, 5.674659655724104, 5.699746169164853, 5.943116452503206, 37.53475835775574]
[7.20792916e-02 1.784

In [39]:
# Testing frequent queue
theta_frequent = {
    'theta_1': 8,
    'theta_2': 16,
    'theta_3': 0.15
}
v_sim2, y_sim2 = simulate_v(theta_frequent, len(Frequent), Frequent)
print(v_sim2)
print(y_sim2)
x_sim2 = np.cumsum(y_sim2)
print(x_sim2)
print(np.cumsum(Frequent))

[0.028927620898199412, 0.05091552625340991, 0.10017372368104926, 0.20763743851961475, 0.3974169564750658, 0.4358185714986952, 0.4781496241370815, 0.48439599253208143, 0.7868500563236643, 0.7967462010271718, 0.8525009248904154, 1.047628016918336, 1.1052740655234627, 1.1495493891104593, 1.180101837108091, 1.259049401436398, 1.284209123417995, 1.3247637374286154, 1.3790650928777324, 1.8196357451680336, 2.0113771112651, 2.0585394454831105, 2.213858445598961, 2.2530854561638183, 2.2584153248948056, 3.2665418045770407, 3.28173839290064, 3.4792510590954078, 3.5107713444431528, 3.5306427414725503, 3.698786465751872, 3.7268724074697617, 3.9305207367152932, 4.181759031317622, 4.184098716284872, 4.188031210647479, 4.218684564319899, 4.229354638269926, 4.241298312664041, 4.366453888180349, 4.534570828850705, 4.874695761675849, 5.232436030960097, 5.2944150158854635, 5.426676893430494, 5.5671202041213, 6.043870978138851, 6.089182634332088, 6.321761923176145, 6.3447089767941245]
[11.049524035537733, 

# 2.2 Simple Metropolis updates for parameters

Now, after $V_{i}$ and the data are available, we would like to sample from the conditional posterior distribution of $\theta \sim \pi(\theta|v,y) \propto \pi(v, \theta|y) \propto \pi(\theta)P(v_{1}|\theta) \prod_{i=2}^{n}P(v_{i}|v_{i-1}, \theta) \prod_{i=1}^{n}P(y_{i}|v_{i}, x_{i-1}, \theta)$.

We reparameterize $\theta$ to $\eta$, where $\eta = (\eta_{1}, \eta_{2}, \eta_{3}) = (\theta_{1}, \theta_{2}-\theta_{1}, log(\theta_{3}))$.

Then, we choose a symmetric proposal density $q(.|\eta)$ : $q(x|x^{̃}) ≡ q(x^{̃}|x)$. Then, we generate proposal $\eta^{*} \sim q(\eta^{*}|\eta)$, and compute acceptance probability

$\large a = min(1, \frac{\pi(\eta^{*}|v,y)}{\pi(\eta|v,y)})$.

We use a normal proposla with independent coordinates centered around current value of $\eta$. If proposed value for $\eta_{1}$ or $\eta_{2}$ is outside of its range, we immediately reject the proposal.

The logarithm of posterior simplifies to
$\large log \pi(v, \theta|y) = log(\pi(\theta)) + nlog(\theta_{3}) - \theta_{3}v_{n} - nlog(\theta_{2} - \theta_{1})$

With these constraints (due to the bounds on original parameters $\theta$):

$\theta_{2} > \theta_{1}$

$\theta_{1} \le y_{1}-v_{1}$

$\theta_{2} \ge y_{1}-v_{1}$

$\theta_{1} \le min(y_{i} - max(0, v_{i}-x_{i-1}))$ for $i \ge 2$

$\theta_{2} \ge max(y_{i} - max(0, v_{i} - x_{i-1}))$ for $i \ge 2$

Note that the paper places a $Uniform(0, 10)$ prior on $\theta_{1}$ and on $\theta_{2} - \theta_{1}$, and a $Uniform(0, 1/3)$ prior on $\theta_{3}$. So, the joint prior $\pi(\theta)$ for $\theta$ is:

$\large \pi(\theta) \propto I(\theta_{1} \in [0,10]) I(\theta_{2}-\theta_{1} \in [0,10])I(\theta_{3} \in [0,1/3])$

With the $\eta$ parameterization, priors on $\eta_{1}$ and $\eta_{2}$ remain the same, and $\eta_{3}$ is now proportional to $exp(\eta_{3})$ on $(-∞, log(1/3))$ and $0$ otherwise.

In [92]:
# theta should be a dictionary
def log_posterior(theta, v, y):
  n = len(v)
  indicator_constraint = all([0 <= theta['theta_1'] <= 10,
                              0 <= theta['theta_2'] - theta['theta_1'] <= 10,
                              0 <= theta['theta_3'] <= 1/3])
  constraint_1 = theta['theta_2'] > theta['theta_1']
  constraint_2 = theta['theta_1'] <= y[0] - v[0]
  constraint_3 = theta['theta_2'] >= y[0] - v[0]
  constraint_4 = theta['theta_1'] <= min([y[i] - max(0, v[i] - np.sum(y[:i])) for i in range(1, len(y))])
  constraint_5 = theta['theta_2'] >= max([y[i] - max(0, v[i] - np.sum(y[:i])) for i in range(1, len(y))])
  print(constraint_)
  if all([indicator_constraint, constraint_1, constraint_2, constraint_3, constraint_4, constraint_5]) == True:
    log_posterior = np.log(float(indicator_constraint)) +\
    n * np.log(theta['theta_3']) -\
     theta['theta_3'] * v[-1] -\
      n * np.log(theta['theta_2'] - theta['theta_1'])
  else:
    log_posterior = -np.inf
  return log_posterior

In [93]:
def metropolis_simple(eta_sd, theta_dict, v, y, n_iterations, n_burn_in):
  MC_chain = []
  for i in range(n_iterations + n_burn_in):
    # gibbs sampler first, and then calculate posterior
    samples = gibbs_sampler_queue(y, len(y), v, theta_dict)
    if i == 0:
      # center first draw on the expectations of the priors on thetas/etas
      eta_init = [float(min(y)), 10/2, float(np.log(1/6))]
      proposal = scipy.stats.multivariate_normal(mean = eta_init,
                                                 cov = np.diag([x ** 2 for x in eta_sd])).rvs(size = 1)
      next_state = proposal
    else:
      current_state = MC_chain[i-1]
      proposal = scipy.stats.multivariate_normal(mean = current_state,
                                                 cov = np.diag([x ** 2 for x in eta_sd])).rvs(size = 1)
      theta_current = {
          "theta_1": float(current_state[0]),
          "theta_2": float(current_state[0]) + float(current_state[1]),
          'theta_3': np.exp(float(current_state[2]))
      }
      theta_proposal = {
          'theta_1': float(proposal[0]),
          'theta_2': float(proposal[1]) + float(proposal[0]),
          'theta_3': np.exp(float(proposal[2]))
      }
      log_posterior_proposal = log_posterior(theta_proposal, samples, y)
      if log_posterior_proposal == -np.inf:
        next_state = current_state
      else:
        log_posterior_current = log_posterior(theta_current, samples, y)
        acceptance_ratio = float(min(1, np.exp(log_posterior_proposal - log_posterior_current)))
        U = float(scipy.stats.uniform().rvs(1))
        if U <= acceptance_ratio:
          next_state = proposal
        else:
          next_state = current_state
    MC_chain.append(next_state)
  # Throw out burn-in samples
  MC_chain = MC_chain[n_burn_in:]
  return MC_chain

In [94]:
eta_intermediate = [0.0764, 0.1093, 0.1441]
test_run = metropolis_simple(eta_intermediate, theta_intermediate, v_sim, Intermediate, 100, 0)

True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf
True
-inf


In [86]:
len(test_run)
eta_1 = [x[0] for x in test_run]
eta_2 = [y[1] for y in test_run]
eta_3 = [z[1] for z in test_run]

In [89]:
np.mean(eta_2)
statistics.variance(eta_2)

np.float64(0.0)